1\. **PCA on 3D dataset**

* Generate a dataset simulating 3 features, each with N entries (N being ${\cal O}(1000)$). Each feature is made by random numbers generated according the normal distribution $N(\mu,\sigma)$ with mean $\mu_i$ and standard deviation $\sigma_i$, with $i=1, 2, 3$. Generate the 3 variables $x_{i}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues using the eigendecomposition of the covariance matrix
* Find the eigenvectors and eigenvalues using the SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained
* Redefine the data according to the new basis from the PCA
* Plot the data, in both the original and the new basis. The figure should have 2 rows (the original and the new basis) and 3 columns (the $[x_0, x_1]$, $[x_0, x_2]$ and $[x_1, x_2]$ projections) of scatter plots.

In [19]:
import numpy as np
import pandas as pd
import matplotlib as plt

L = 1000
x1 = np.random.normal(0, 1, L)
x2 = np.random.normal(0, 3, L)
x3 = 2*x1 + x2
dict = {'Feature 1': x1, 'Feature 2': x2, 'Feature 3': x3}
data = pd.DataFrame(dict)
autovalori, autovettori = np.linalg.eig(data.cov())
print("Autovalori calcolati dalla matrice di covarianza:")
print(autovalori)
print("Autovettori calcolati dalla matrice di covarianza:")
print(autovettori)
print()
valori = data.values
U, sigma, Vt = np.linalg.svd(valori)
eigenvalues = sigma**2
eigenvectors = Vt.T
print("Autovalori calcolati con SVD:")
print(sigma**2)
print("Autovettori calcolati con SVD:")
print(eigenvectors)
print()
#I valori sono simili ma non uguali
tot_variance = np.sum(eigenvalues)
explained = eigenvalues / tot_variance
print("Percentuale del dataframe spiegata dagli autovalori:", explained*100, "\n")
#riduco la dimensione degli autovettori
obiettivo = 0.99
cumulative_variance = np.cumsum(explained)
dimens = np.argmax(cumulative_variance >= obiettivo) + 1
reduced = eigenvectors[:, :dimens]
print("I nuovi autovettori per il 99% di variabilità sono:")
print(reduced)
print("Con dimensioni:", reduced.shape)
#Sistemo il dataframe di conseguenza
data99 = pd.DataFrame(data.values.dot(reduced), columns=["Component1", "Component2"])
print(data99)

Autovalori calcolati dalla matrice di covarianza:
[ 1.81414163e+01 -1.72431643e-15  2.60548044e+00]
Autovettori calcolati dalla matrice di covarianza:
[[-0.09174656  0.81649658 -0.57001395]
 [-0.6063751   0.40824829  0.68238008]
 [-0.78986822 -0.40824829 -0.45764782]]

Autovalori calcolati con SVD:
[1.81243079e+04 2.60574977e+03 7.44699654e-28]
Autovettori calcolati con SVD:
[[-0.09180985 -0.57000376  0.81649658]
 [-0.60629932  0.68244741  0.40824829]
 [-0.78991903 -0.45756011 -0.40824829]]

Percentuale del dataframe spiegata dagli autovalori: [8.74300891e+01 1.25699109e+01 3.59236654e-30] 

I nuovi autovettori per il 99% di variabilità sono:
[[-0.09180985 -0.57000376]
 [-0.60629932  0.68244741]
 [-0.78991903 -0.45756011]]
Con dimensioni: (3, 2)
     Component1  Component2
0     -1.218680   -1.687183
1      0.853343    1.211166
2     -1.801004    3.416497
3     -3.896160    1.270306
4      0.365968    0.819561
..          ...         ...
995   10.886281    0.343916
996   -1.053998   -3

2\. **PCA on a nD dataset**

* Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normally distributed, with a standard deviation much smaller (e.g. a factor 20) than those used to generate the $x_1$ and $x_2$. Repeat the PCA procedure and compare the results with what you have obtained before.

3\. **Optional**: **PCA on the MAGIC dataset**

Perform a PCA on the magic04.data dataset.

In [ ]:
# get the dataset and its description on the proper data directory
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P data/
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P data/ 